# Topic Modeling
---
#### Imports

In [155]:
# processing
from operator import methodcaller
import csv
import re
import numpy as np
import pandas as pd
from pprint import pprint

# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

#sci-kit
from sklearn import feature_extraction

C:\Users\thepe\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


#### Processing

In [144]:
blacklist = [t.strip() for t in next(csv.reader(open("blacklist.csv", 'r')))]

inPath = "input.csv"
outPath = "out.csv"
wordBound = 10
charBound = 70

inFile = open(inPath, 'r')
inReader = csv.reader(inFile)

outFile = open(outPath, 'w')
outWriter = csv.writer(outFile)


docTokens = dict()


next(inReader)
for inRow in inReader:

    charDist = int(inRow[0])
    wordDist = int(inRow[1])

    if wordDist < wordBound and charDist < charBound:

        #predTerm, subTerm, objTerm = map(methodcaller("split", ":"), inRow[2:5])
        #allTerms = predTerm + subTerm + objTerm

        subTerm, objTerm = map(methodcaller("split", ":"), inRow[3:5])
        
        sub = "_".join([t for t in subTerm if re.match(r'[^\W\d]*$', t) and not t in blacklist])
        obj = "_".join([t for t in subTerm if re.match(r'[^\W\d]*$', t) and not t in blacklist])
        
        tokens = list()
        if not sub in blacklist and not obj in blacklist and len(sub) > 0 and len(obj) > 0:
            tokens = [sub, obj]

            docID = inRow[5]

            if docID in docTokens:
                docTokens[docID] += tokens
            else:
                docTokens[docID] = tokens
docIDs = list(docTokens.keys())
data = list(docTokens.values())

#### Model

In [3]:
id2word = corpora.Dictionary(data)
texts = data

corpus = [id2word.doc2bow(text) for text in texts]

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=40, 
                                       random_state=100,
                                       update_every=1,
                                       chunksize=20,
                                       passes=20,
                                       alpha='auto',
                                       per_word_topics=True)


pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(21,
  '0.195*"molecular_hydrogen" + 0.065*"one_solution" + 0.060*"inversion_dimer" '
  '+ 0.034*"diffraction_spot" + 0.034*"basal_diffraction_spot_intensity" + '
  '0.034*"cone_series" + 0.034*"basal_diffraction_spot" + '
  '0.028*"piperazine_ring" + 0.020*"double_bond" + 0.020*"c_double_bond"'),
 (36,
  '0.124*"structure_solution" + 0.101*"water_molecule" + '
  '0.062*"molecular_skeleton" + 0.062*"water_molecule_involve_n_hydrogen_bond" '
  '+ 0.038*"vv_atom" + 0.038*"molecular_salt" + 0.038*"basal_plane" + '
  '0.025*"octahedral_coordination" + 0.019*"title_complete_cation" + '
  '0.019*"strongly_distort_octahedron"'),
 (19,
  '0.078*"kda_protein" + 0.078*"da_minus" + 0.052*"sluggish_transformation" + '
  '0.052*"vm_value" + 0.052*"obtain_agreement" + 0.027*"cuii_atom" + '
  '0.026*"atp_complex_erk_detail_structural_analysis" + '
  '0.026*"detail_structural_analysis" + 0.026*"dihydroorotate_dehydrogenase" + '
  '0.026*"putative_dihydroorotate_dehydrogenase"'),
 (6,
  '0.101*"import

#### Visualize

In [156]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.show(vis)

C:\Users\thepe\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [03/Jul/2019 22:08:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2019 22:08:27] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2019 22:08:27] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2019 22:08:27] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2019 22:08:27] code 404, message Not Found
127.0.0.1 - - [03/Jul/2019 22:08:27] "GET /favicon.ico HTTP/1.1" 404 -



stopping Server...


In [139]:

from sklearn.feature_extraction.text import TfidfVectorizer

newDocs = list()
for doc in data:
    newDocs.append(" ".join(doc))


#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.05, max_features=200000,
                                 min_df=0.000, stop_words='english',
                                 use_idf=True, tokenizer=None, ngram_range=(1, 1))

tfidf_matrix = tfidf_vectorizer.fit_transform(newDocs) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

(923, 2200)


In [140]:
terms = tfidf_vectorizer.get_feature_names()

from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [161]:
from sklearn.cluster import KMeans

num_clusters = 20

km = KMeans(n_clusters=num_clusters)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

Wall time: 342 ms


In [183]:
import sys
import urllib.request
from urllib.error import HTTPError


BASE_URL = 'http://dx.doi.org/'

def getTitle(doi):
    url = BASE_URL + doi
    req = urllib.request.Request(url)
    req.add_header('Accept', 'application/x-bibtex')
    try:
        with urllib.request.urlopen(req) as f:
            bibtex = f.read().decode()
        start = bibtex.find("title = {")
        end = bibtex.find("},", start)
        return bibtex[start + 9:end]
        
        
    except HTTPError as e:
        if e.code == 404:
            return('DOI not found.')
        else:
            return('Service unavailable.')

In [162]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()


print(clusters)

[7, 7, 7, 7, 7, 7, 7, 7, 13, 1, 7, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 7, 7, 7, 1, 7, 7, 1, 7, 19, 12, 7, 7, 7, 1, 7, 7, 7, 7, 7, 1, 7, 9, 1, 7, 1, 1, 7, 7, 1, 7, 7, 7, 3, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 16, 1, 7, 0, 1, 7, 7, 7, 6, 7, 1, 3, 7, 17, 7, 7, 7, 7, 7, 9, 7, 1, 2, 7, 7, 7, 1, 7, 10, 1, 7, 7, 7, 1, 7, 7, 11, 7, 1, 7, 6, 7, 7, 0, 8, 7, 1, 1, 7, 7, 7, 1, 7, 10, 7, 7, 1, 7, 1, 7, 1, 7, 7, 7, 7, 7, 7, 7, 10, 7, 1, 7, 7, 19, 7, 7, 1, 14, 7, 1, 7, 1, 7, 7, 7, 7, 7, 1, 16, 7, 7, 7, 7, 7, 7, 7, 1, 7, 1, 7, 1, 1, 7, 7, 1, 1, 7, 7, 13, 7, 7, 7, 1, 7, 1, 1, 1, 7, 7, 1, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 7, 7, 7, 7, 1, 1, 7, 7, 7, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 9, 7, 7, 7, 7, 7, 7, 7, 7, 7, 5, 7, 7, 1, 7, 7, 7, 7, 1, 9, 7, 7, 5, 7, 7, 7, 7, 7, 7, 0, 1, 14, 7, 7, 0, 1, 7, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 7, 14, 7, 18, 7, 1, 1, 1, 7, 1, 1, 0, 1, 7, 7, 7, 7, 7, 7, 0, 1, 1, 7, 7, 7, 7, 7, 16, 7, 7, 1, 7, 1, 7, 7, 7, 7, 7, 7, 7

In [163]:
documents = {'docID': docIDs, 'text': data, 'cluster': clusters }

frame = pd.DataFrame(documents, index = [clusters] , columns = ['docID','cluster'])

frame['cluster'].value_counts() #number of films per cluster (clusters from 0 to 4)



7     614
1     141
2      31
9      22
14     14
10     13
6      11
5      10
0      10
11     10
17      9
3       7
15      6
4       6
8       5
19      5
16      4
13      3
18      1
12      1
Name: cluster, dtype: int64

In [ ]:
from __future__ import print_function

print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid

outWriter = csv.writer(open("out.csv", 'w'), lineterminator = "\n")


for i in range(num_clusters):
    print("Cluster %d docIDs:" % i, end='')
    
    DOIs = frame.ix[i]['docID'].values.tolist()
    titles = [getTitle(ID) for ID in DOIs]
    print(titles)
    outWriter.writerow(titles)
    print() #add whitespace
    print() #add whitespace
    
print()
print()

Top terms per cluster:

Cluster 0 docIDs:

C:\Users\thepe\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


['Di-2-pyridyl ketonep-aminobenzoylhydrazone hydrate', 'A novel metallo-organically templated pentaborate: acetato[N,N$\\prime$-bis(2-aminoethyl)ethane-1,2-diamine]zinc({II}) 4,4$\\prime$,6,6$\\prime$-tetrahydroxy-2,2$\\prime$-spirobi[cyclotriboroxane](1-)', 'Crystal structure of $\\upbeta$-luffin, a ribosome-inactivating protein, at 2.0{\\hspace{0.25em}}{\\{AA}} resolution', 'Gene design, expression, crystallization and preliminary diffraction analysis of two isolectins from the {fungusCoprinus} cinereus: a model for studying functional diversification of galectins in the same organism and their evolutionary pathways', 'Structural basis of high-order oligomerization of the cullin-3 adaptor {SPOP}', 'Preliminary crystallographic characterization of anin vitroevolved biotin-binding {RNA} pseudoknot', 'Purification and crystallization of Cor a 9, a major hazelnut allergen', 'Cloning, expression, purification and crystallization of an endotoxin-biosynthesis enzyme {fromNeisseria} meningit

C:\Users\thepe\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


['Polarised neutron investigation of iron composite nanoparticles', 'Accuracy in Rietveld quantitative phase analysis of Portland cements', 'Combined synchrotron X-ray and image-correlation analyses of biaxially deformed W/Cu nanocomposite thin films on Kapton', 'A dynamic study of the crystallization of polyethylene from the melt', 'A new method to determine the exact values of the fiber identity period of polyamides', 'Whole-powder-pattern fitting without reference to a structural model: application to X-ray powder diffraction data', 'Structure refinement of {GeO}2 polymorphs at high pressures and temperatures by energy-dispersive spectra of powder diffraction', 'Relation between the optical properties and structure of {KLiSO}4in the room-temperature phase', 'The structure of pumice by neutron diffraction', 'An open-flow cryogenic cooler for single-crystal diffraction experiments', 'Determination of the Cation Distribution in {NiFe}2({PO}4)2 using Resonant X-ray and Neutron Powder Di

...

C:\Users\thepe\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


['The negative-quartet relation from electron-density considerations', 'Separating nucleation and growth in protein crystallization using dynamic light scattering', 'Local structure study of dilute Er in {III}{\\textendash}V semiconductors by fluorescence {EXAFS}', 'Ethyl 2-acetyl-3-anilinobutanoate', '(Acetonitrile-$\\upkappa$N)(3-amino-4-methylbenzenesulfonato-$\\upkappa$N)aqua(triphenylphosphine-$\\upkappa$P)silver(I) hemihydrate', '1-(Benzylideneamino)pyridinum iodide', 'N-[11-(4-Chlorophenyl)-11,12-dihydrobenzo[c]phenanthridin-6-yl]benzamide', '1,3-Bis(4-fluorophenyl)-N,N$\\prime$-(propane-1,3-diylidene)dihydroxylamine', '4-tert-Butylpyridinium chloride{\\textendash}4,4$\\prime$-(propane-2,2-diyl)bis(2,6-dimethylphenol){\\textendash}toluene (2/2/1)', 'Crystallization and preliminary crystallographic analysis of the catechol 2,3-dioxygenase {PheB} {fromBacillus} {stearothermophilusBR}219']


Cluster 6 docIDs:

C:\Users\thepe\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]


['Dinicotinamidium squarate', 'Four 2-amino-6-aryl-4-methoxy-11H-pyrimido[4,5-b][1,4]benzodiazepines: similar molecular structures but different crystal structures', 'Bis[1-(2-hydroxyethyliminomethyl)-2-naphtholato-$\\upkappa$2N,O]nickel({II})', 'Triaquabis(4-formylbenzoato-$\\upkappa$2O,O$\\prime$)cadmium({II}) 3.5-hydrate', '3-Phenyl-1-[2-(3-phenylisoquinolin-1-yl)diselanyl]isoquinoline', '2-(1H-Benzotriazol-1-yl)-1-phenylethanol', '3,5-Bis(4-hydroxyphenyl)-4H-1,2,4-triazol-4-amine monohydrate', '4-Hydrazinylidene-1-methyl-3H-2$\\uplambda$6,1-benzothiazine-2,2-dione', '2-(4-Chloro-1H-indol-3-yl)acetonitrile', 'Ergotaminine', 'Methyl 4-(benzyloxy)-3-methoxybenzoate']


Cluster 7 docIDs:

C:\Users\thepe\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]
